In [1]:
import os
import numpy as np
import pandas as pd
import json
import cv2

import tensorflow as tf
from tensorflow import keras
# import tensorflow_hub as hub

from keras import layers, models, optimizers, regularizers
from keras.applications import MobileNet, EfficientNetB0
from keras.applications.efficientnet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# import matplotlib.pyplot as plt

In [2]:
# test if running the GPU version of tensorflow
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
# loads directory of 3 datasets
dir_train = pd.read_csv('dataset/EuroSAT/train.csv')
dir_valid = pd.read_csv('dataset/EuroSAT/validation.csv')
dir_test = pd.read_csv('dataset/EuroSAT/test.csv')


In [4]:
img_dir = pd.concat([dir_train, dir_valid], ignore_index=False)
img_dir = img_dir.iloc[:,1:-1].reset_index().drop(['index'], axis=1)
print(img_dir.shape)
# img_dir = img_dir.iloc[:1000, :] # limit sample size when testing
img_dir

(24300, 3)


,Filename,Label,ClassName
0,AnnualCrop/AnnualCrop_142.jpg,0,AnnualCrop
1,HerbaceousVegetation/HerbaceousVegetation_2835...,2,HerbaceousVegetation
2,PermanentCrop/PermanentCrop_1073.jpg,6,PermanentCrop
3,Industrial/Industrial_453.jpg,4,Industrial
4,HerbaceousVegetation/HerbaceousVegetation_1810...,2,HerbaceousVegetation
...,...,...,...
24295,SeaLake/SeaLake_1943.jpg,9,SeaLake
24296,AnnualCrop/AnnualCrop_211.jpg,0,AnnualCrop
24297,Industrial/Industrial_1428.jpg,4,Industrial
24298,AnnualCrop/AnnualCrop_2571.jpg,0,AnnualCrop


In [5]:
# Load images and labels/classes
images = []
classes = []
# labels.typeof()

base_path = 'dataset/EuroSAT/'
for index, row in img_dir.iterrows():
    img_path = os.path.join(base_path, row['Filename'])
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img)
    classes.append(row['ClassName'])


In [6]:
# Normalize images
images = np.array(images)/255.0
labels = pd.get_dummies(classes).values

In [7]:
print(images.shape)
images[0].max()

(24300, 224, 224, 3)


1.0

In [8]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Set up the data augmentation
train_data_generator = ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)
valid_data_generator = ImageDataGenerator()

In [9]:
def get_model(res=224, num_classes=10):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(res, res, 3))
    base_model.trainable = False
    
    model = keras.Sequential(
        [
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dense(256, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
            keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )
    return model

model_1 = get_model()
model_1.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               327936    
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                2

In [10]:
# Compile the model
model_1.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10
validation_steps = len(X_test) // batch_size

history = model_1.fit(train_data_generator.flow(X_train, y_train, batch_size=batch_size), validation_data=valid_data_generator.flow(X_test, y_test, batch_size=batch_size), validation_steps=validation_steps, epochs=epochs)

Epoch 1/10
608/608 [==============================] - 166s 253ms/step - loss: 2.7434 - accuracy: 0.1093 - val_loss: 2.7450 - val_accuracy: 0.0962
Epoch 2/10
608/608 [==============================] - 159s 261ms/step - loss: 2.4447 - accuracy: 0.1128 - val_loss: 2.7470 - val_accuracy: 0.1120
Epoch 3/10
608/608 [==============================] - 152s 250ms/step - loss: 2.3564 - accuracy: 0.1240 - val_loss: 2.6530 - val_accuracy: 0.1097
Epoch 4/10
608/608 [==============================] - 178s 292ms/step - loss: 2.3317 - accuracy: 0.1243 - val_loss: 2.5200 - val_accuracy: 0.0966
Epoch 5/10
608/608 [==============================] - 178s 293ms/step - loss: 2.3191 - accuracy: 0.1260 - val_loss: 3.1254 - val_accuracy: 0.0964
Epoch 6/10
 15/608 [..............................] - ETA: 2:30 - loss: 2.3138 - accuracy: 0.1500

KeyboardInterrupt: 

In [21]:
model_1.save("model_MobileNet_nFull_epoch10_batch32")

INFO:tensorflow:Assets written to: model_MobileNet_nFull_epoch10_batch32\assets


INFO:tensorflow:Assets written to: model_MobileNet_nFull_epoch10_batch32\assets


In [22]:
## if want to load later on: 
from tensorflow.keras.models import load_model
model_1 = load_model("model_MobileNet_nFull_epoch10_batch32")


# ## Alternative way - only save the weights
# model_loaded = get_model()
# model_loaded.load_weights("model_EfficientNetB0_nFull_epoch10_batch32_weights.h5")

In [19]:
X_test[0].shape

(224, 224, 3)

In [23]:
# Evaluate the model
batch_size = 32  # Choose a smaller batch size according to your GPU memory capacity
y_pred_batches = []

for i in range(0, len(X_test), batch_size):
    batch_pred = model_1.predict(X_test[i:i + batch_size])
    y_pred_batches.append(batch_pred)

y_pred = np.concatenate(y_pred_batches, axis=0)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# y_pred = model_1.predict(X_test)
# y_pred_classes = np.argmax(y_pred, axis=1)
# y_true = np.argmax(y_test, axis=1)

1/1 [==============================] - 0s 272ms/step


In [24]:
print(classification_report(y_true, y_pred_classes))
print(confusion_matrix(y_true, y_pred_classes))

              precision    recall  f1-score   support

           0       0.97      0.89      0.93       551
           1       0.93      0.92      0.93       547
           2       0.87      0.92      0.89       535
           3       0.79      0.93      0.86       459
           4       0.98      0.95      0.96       453
           5       0.87      0.91      0.89       371
           6       0.90      0.87      0.89       456
           7       0.96      0.98      0.97       502
           8       0.94      0.79      0.86       469
           9       0.94      0.98      0.96       517

    accuracy                           0.92      4860
   macro avg       0.92      0.91      0.91      4860
weighted avg       0.92      0.92      0.92      4860

[[488   0   1  12   0  11  22   0  11   6]
 [  1 503  11   1   0   8   0   0   0  23]
 [  0  13 490  10   0   8  11   3   0   0]
 [  4   1   2 429   2   5   4   2  10   0]
 [  0   0   0   8 430   0   4  11   0   0]
 [  0  11  14   5   0 338 

In [ ]:
# # Evaluate the model
# batch_size = 32  # Choose a smaller batch size according to your GPU memory capacity
# y_pred_batches = []

# for i in range(0, len(X_test), batch_size):
#     batch_pred = model_loaded.predict(X_test[i:i + batch_size])
#     y_pred_batches.append(batch_pred)

# y_pred = np.concatenate(y_pred_batches, axis=0)
# y_pred_classes = np.argmax(y_pred, axis=1)
# y_true = np.argmax(y_test, axis=1)

# # y_pred = model_1.predict(X_test)
# # y_pred_classes = np.argmax(y_pred, axis=1)
# # y_true = np.argmax(y_test, axis=1)

##### If you want to save the model: 

In [ ]:
# model_1.save("model_1")

# ## if want to load later on: 
# from tensorflow.keras.models import load_model
# loaded_model = load_model("model_1")

## Draft / Archive (Plz disregard)

In [ ]:
# import matplotlib.pyplot as plt # somehow this just doesnt work

In [ ]:
# testing/debugging
# print(classes[0])
# images[0].shape

In [ ]:
# ## Use this when running for 1st time
# MODEL_PATH = "https://tfhub.dev/sayakpaul/convnext_base_21k_1k_224_fe/1"
# model = hub.load(MODEL_PATH)
# # saved_model_path = 'model/'
# # tf.saved_model.save(model, saved_model_path)

# # ## Use this for subsequent runs when model already loaded locally
# # saved_model_path = 'model/'
# # model = tf.saved_model.load(saved_model_path)